In [2]:
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', 'raise')
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

# Suppress warning about mixed data types in columns (change this later?)
import warnings
warnings.filterwarnings('ignore')

# Import Some of the useful defined functions
import useful_fun as uf
# object with different useful regions defined
regions = uf.Regions()

In [3]:
# Extract the the zipped csv data if it already hasn't been extracted 
import os.path
import zipfile

if not os.path.isfile('./data/LiTS_III_2016.csv'):
    with zipfile.ZipFile("./data/LiTS_III_2016.zip","r") as zip_ref:
        zip_ref.extractall("./data/")
else:
    print('File already present')

File already present


In [4]:
# Reading the dataframe
lits_2016_full = pd.read_csv('./data/LiTS_III_2016.csv') # data in the repo folder linux
# lits_2016 = pd.read_csv('LiTS III.csv')

I will try and use the following questions to make k-NN classifier for determening political / market inclination.

age  
gender  
country  
q223 - Monthly Income  
q204 - Internet Acess (Yes / No)  
q109 - Education Level (1 - 8)  
* q109_1 - primary responent

q203 - Books Growing Up (1 - 5)  
q403 - Trust in People (1 - 5)  
q428 - Risktaking (1-10)  
q901 - State of Health (1-5)  
q904 - sources of information (1-7)  
* radio / TV - a  
* social media - g  

905 - hours of TV watched yesterday.  


In [5]:
# Select only the columns we're interested in (can increase this as we go along)
good_cols = [
'country',#     'Country',
'gender_pr',#   'Gender',
'q204',#        'Internet Acess',
'age_pr',#      'Age',
'q223',#        'Monthly Income',
'q109_1',#      'Education Level',
'q203',#        'Books Growing Up',
'q403',#        'Trust in People',
'q428',#        'Risktaking',
'q901',#        'State of Health',
'q904a',#       'TV News',
'q904g',#       'Social Media News',
'q905',#        'TV Time Yesterday',
'q412',#        'Political System',
'q411',#        'Economy Type',
]
lits_2016_selected = lits_2016_full[good_cols]

# Give the columns new names
good_names = [
'Country',
'Gender',
'Internet Acess',
'Age',
'Monthly Income',
'Education Level',
'Books Growing Up',
'Trust in People',
'Risktaking',
'State of Health',
'TV News',
'Social Media News',
'TV Time Yesterday',
'Political System',
'Economy Type',
]
lits_2016_selected.columns = good_names
lits_2016_selected.head()

,Country,Gender,Internet Acess,Age,Monthly Income,Education Level,Books Growing Up,Trust in People,Risktaking,State of Health,TV News,Social Media News,TV Time Yesterday,Political System,Economy Type
0,Albania,2,3,47,-99,2,1,1.0,3.0,4.0,1.0,1.0,2.0,1.0,1.0
1,Albania,1,2,58,50000,2,1,1.0,3.0,3.0,6.0,6.0,5.0,1.0,1.0
2,Albania,1,3,40,-99,7,1,1.0,2.0,3.0,7.0,2.0,5.0,2.0,2.0
3,Albania,1,2,58,30000,2,1,1.0,2.0,3.0,2.0,1.0,2.0,2.0,2.0
4,Albania,2,1,28,-97,7,2,1.0,2.0,5.0,7.0,7.0,9.0,3.0,2.0


Now let's clean up the data  

Remove the following:  
* -91 - difficult to say
* -99 - refusal
* -97 - don't know
* -98 - not applicable

In [6]:
exclusion_criteria = [np.nan, -97, -99, -98, -91]
meassure = lits_2016_selected.iloc[:,1:].columns

lits_2016_selected_clean = uf.exclude_values(lits_2016_selected,meassure,exclusion_criteria)

In [7]:
percentage_after_cleaning = (lits_2016_selected_clean.shape[0]/lits_2016_selected.shape[0])*100
print('The percentage of the rows after cleaning {:2.2f} %, that is {} participants'.format(percentage_after_cleaning, lits_2016_selected_clean.shape[0]))

The percentage of the rows after cleaning 64.04 %, that is 32791 participants


Normalize Monthly income by the median in the country

In [8]:
# Median income values in each country
byIncome_mean = lits_2016_selected_clean.groupby('Country').median()['Monthly Income']

df = lits_2016_selected_clean

# Normalize by the median value of monthly income in each country
for country in byIncome_mean.index:
    df.loc[df['Country']==country,'Monthly Income'] = (df.loc[df['Country']==country,'Monthly Income']/byIncome_mean[country])*100


Adjust answers about internet acess

In [9]:
# 1 - Yes, 2,3,4 - Differnet variations of No

df.loc[df['Internet Acess']>1,'Internet Acess']= 2

### Political Preference / Economy Type classification  
#### Market v Planned economy
q411 (p23 - questionaire)
* 1 - **Capitalist** - A market economy is preferable to any other form of economic system.
* 2 - **Social** - Under some circumstances, a planned economy may be preferable to a market economy.
* 3 - **Indiferent** - For people like me, it does not matter whether the economic system is organised as a market economy or as a planned economy.
* -97 - Don't know.  

#### Democratic v Authoritarian system
q412 (p23 - questionaire)
* 1 - **Democratic** - Democracy is preferable to any other form of political system.
* 2 - **Authoritarian** - Under some circumstances, an authoritarian government may be preferable to a democratic one.
* 3 - **Indiferent** - For people like me, it does not matter whether a government is democratic or authoritarian.
* -97 - Don't know.  


### I will try and use the columns 'Age' to 'TV Time Yesterday' to try and classify inclination towards Democratic or Authoritarian system.

Split the dataframe into training and test sets.

In [10]:
# training_proportion = 11/16
training_proportion = 8/10
participants = len(lits_2016_selected_clean)
num_train = int(round(participants * training_proportion))
# Shuffle the dataframe
lits_2016_selected_clean_shuffled = lits_2016_selected_clean.sample(frac=1,replace=False)

# Assign parts of the shuffled dataframe to training and test sets
train_lits_2016 = lits_2016_selected_clean_shuffled.iloc[:num_train,:].copy()
test_lits_2016 = lits_2016_selected_clean_shuffled.iloc[num_train:,:].copy()
print("Training set:\t", len(train_lits_2016), "examples")
print("Test set:\t", len(test_lits_2016), "examples")

Training set:	 26233 examples
Test set:	 6558 examples


44% - for Political system on the basis of Age - TV Time Yesterday  
45% - for Political system on the basis of Education Level - TV Time Yesterday

In [11]:
# sel_test_feat = test_lits_2016.loc[:,'Age':'Political System']
# sel_train_feat = train_lits_2016.loc[:,'Age':'Political System']
sel_test_feat = test_lits_2016.loc[:,'Education Level':'Political System']
sel_train_feat = train_lits_2016.loc[:,'Education Level':'Political System']

In [12]:
sel_test_feat.head()

,Education Level,Books Growing Up,Trust in People,Risktaking,State of Health,TV News,Social Media News,TV Time Yesterday,Political System
15002,5,5,4.0,10.0,3.0,6.0,7.0,5.0,3.0
41378,2,1,3.0,8.0,2.0,1.0,1.0,1.0,3.0
24485,6,3,1.0,10.0,2.0,6.0,7.0,2.0,1.0
38119,5,5,2.0,2.0,3.0,5.0,1.0,3.0,1.0
4035,4,1,3.0,4.0,3.0,4.0,2.0,2.0,3.0


In [15]:
sel_train_feat_with_dist = uf.distances(sel_train_feat, sel_test_feat.iloc[0],'Political System')
sel_train_feat_with_dist

,Education Level,Books Growing Up,Trust in People,Risktaking,State of Health,TV News,Social Media News,TV Time Yesterday,Political System,Distance
18463,3,2,4.0,6.0,2.0,1.0,7.0,0.0,1.0,8.944272
38589,7,2,4.0,3.0,2.0,5.0,1.0,1.0,1.0,10.770330
51069,5,1,1.0,10.0,2.0,6.0,1.0,1.0,1.0,8.831761
5410,5,3,3.0,4.0,3.0,1.0,7.0,3.0,1.0,8.366600
46745,4,1,4.0,6.0,3.0,5.0,5.0,4.0,1.0,6.244998
...,...,...,...,...,...,...,...,...,...,...
42454,3,2,2.0,2.0,2.0,1.0,1.0,3.0,1.0,12.124356
7016,4,3,1.0,8.0,3.0,6.0,6.0,7.0,3.0,4.795832
11524,4,2,4.0,8.0,2.0,5.0,1.0,2.0,1.0,7.810250
31328,8,5,4.0,5.0,2.0,6.0,7.0,0.0,1.0,7.745967


In [17]:
# What is the actual example classification
print('First test row classification: {}'.format(sel_test_feat.iloc[0,-1]))

# Prediction based on the classification
# print('5-NN classification: {}'.format(predict_nn(sel_test_feat.iloc[0])))
print('5-NN classification: {}'.format(uf.predict_nn(sel_test_feat.iloc[0], sel_train_feat, 15, 'Political System')))

First test row classification: 3.0
5-NN classification: 1.0


In [18]:
# Run the classifier on all the test values
# sel_test_feat['Guesses'] = sel_test_feat.apply(predict_nn, axis = 1)
sel_test_feat['Guesses'] = sel_test_feat.apply(uf.predict_nn,args=(sel_train_feat, 15, 'Political System'), axis = 1)
sel_test_feat.head(10)

,Education Level,Books Growing Up,Trust in People,Risktaking,State of Health,TV News,Social Media News,TV Time Yesterday,Political System,Guesses
15002,5,5,4.0,10.0,3.0,6.0,7.0,5.0,3.0,1.0
41378,2,1,3.0,8.0,2.0,1.0,1.0,1.0,3.0,1.0
24485,6,3,1.0,10.0,2.0,6.0,7.0,2.0,1.0,1.0
38119,5,5,2.0,2.0,3.0,5.0,1.0,3.0,1.0,2.0
4035,4,1,3.0,4.0,3.0,4.0,2.0,2.0,3.0,2.0
24839,3,1,1.0,2.0,3.0,5.0,1.0,4.0,2.0,3.0
14515,4,1,4.0,7.0,4.0,1.0,1.0,14.0,2.0,1.0
17038,8,3,2.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0
40252,7,4,1.0,6.0,3.0,1.0,7.0,0.0,3.0,1.0
50511,5,1,4.0,5.0,2.0,6.0,1.0,0.0,1.0,1.0


In [19]:
proportion_correct = np.count_nonzero(sel_test_feat['Guesses']== sel_test_feat['Political System'])
proportion_correct = proportion_correct/len(sel_test_feat) 
print('The 5-NN classifier using 8 features is {:2.4f} % accurate'.format(proportion_correct*100))

The 5-NN classifier using 8 features is 46.4166 % accurate


Let's have a look at the different features